In [ ]:
%pip -q install google-genai

In [ ]:
# Configura a API Key do Google Gemini

import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [ ]:
# Configura o cliente da SDK do Gemini

from google import genai

client = genai.Client()

MODEL_ID_FAST = "gemini-2.0-flash"
MODEL_ID_THINKING = "gemini-2.5-flash-preview-04-17"

In [ ]:
# Instalar Framework ADK de agentes do Google ################################################
!pip install -q google-adk

In [ ]:
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types  # Para criar conteúdos (Content e Part)
from datetime import date
from google.colab import files # para poder selecionar arquivos de sua máquina e enviar para o colab
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, HTML, Markdown # Para exibir texto formatado no Colab
import requests # Para fazer requisições HTTP
import time
import warnings

warnings.filterwarnings("ignore")

In [ ]:
# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, contents: types.Content) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    #content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=contents):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

In [ ]:
# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [ ]:
##########################################
# --- Agente 1: Analisador de arquivos --- #
##########################################
def agente_analisador_de_arquivos(uploaded_files: dict):
    analisador_de_arquivos = Agent(
        name="agente_analisador_de_arquivos",
        model=MODEL_ID_THINKING,
        instruction="""
        Você é um assistente de declaração de imposto de renda no Brasil e que conhece muito de ativos, como ações, fundos, FII's, ETF's e outros ativos.
        Você tem amplo conhecimento contábil, principalmente com temas relacionados a declaração de imposto de renda pessoa física (IRPF) .
        Você receberá uma lista de arquivos e vai processá-los identificando dados como: a data da operação, qual foi o ativo negociado, qual o valor unitário
        e qual o valor total da operação.

        Ponto muito importante sobre a informação de "Tipo de Operação" (coluna com título "C/V"), na grande maioria dos documentos de notas de corretagem é uma coluna com título "C/V" ,
        onde "C" é para uma operação de (Compra) e "V" para uma operação de (Venda)

        Não tente aglutinar operações, se em um mesmo arquivo existem 2 operações para o mesmo ativo, apresente 2 operações

        As operações deverão aparecer ordenadas por ativo e por data, considerando como primeiro critério o ativo e segundo critério as datas, ambos os critérios de forma crescente.

        Quero que apresente os dados no formato abaixo e responda de forma bem sucinta, focando apenas em dados do formato abaixo:
        Após processar e analisar os dados você criará uma lista de dados com o sequinte formato:

        Ativo 1
          - (Data da operação) |  (C/V (tipo de operação)) | (Valor unitário) | (Quantidade negociada) | (Valor total)


        Ativo 2
          - (Data da operação) |  (C/V (tipo de operação)) | (Valor unitário) | (Quantidade negociada) | (Valor total)


        Ativo 3
          - (Data da operação) |  (C/V (tipo de operação)) | (Valor unitário) | (Quantidade negociada) | (Valor total)


        As operações deverão aparecer ordenadas por data, do mais antigo para o mais recente.
        """,
        description="Agente que analisa arquivos de notas de corretagem"
    )

    parts = []
    print("Arquivos enviados para o Gemini:")
    for filename in filenames:
      try:
        print(f"Nome do arquivo: {filename}")
        file = client.files.upload(file=filename)
        parts.append( types.Part.from_uri(
              file_uri=file.uri,
              mime_type=file.mime_type,
            )
        )
      except FileNotFoundError:
        print(f"Arquivo não encontrado: {filename}")
      except Exception as e:
        print(f"Erro ao carregar o arquivo {filename}: {e}")


    entrada_do_agente = types.Content(
        role="user",
        parts=parts,
    )

    #print(entrada_do_agente)

    # Executa o agente
    operacoes = call_agent(analisador_de_arquivos, entrada_do_agente)
    return operacoes

In [ ]:
################################################
# --- Agente 2: Agente que processa dados das operações--- #
################################################
def agente_de_processamento_de_dados_de_operacoes(operacoes):
    agente_de_processamento_de_operacoes = Agent(
        name="agente_de_processamento_de_operacoes",
        model=MODEL_ID_THINKING,
        # Inserir as instruções do Agente de processamento de operações #################################################
        instruction="""
        Você é um assistente de declaração de imposto de renda no Brasil. Você tem amplo conhecimento contábil, principalmente com
        temas relacionados a declaração de imposto de renda pessoa física (IRPF). Você tem amplo conhecimento em cálculo
        de preço médio de ativos negociados.

        Você receberá uma lista de operações de compra e venda de ativos e deverá realizar o cálculo de custo médio de cada ativo.
        Ao final do processamento gere uma lista com os ativos e seus respectivos custos médios.
        Também apresente a quantidade e o valor total de cada ativo.
        Quero que apresente os dados no formato abaixo:

        Ativo 1
          - Quantidade
          - Valor total
          - Custo médio


        Ativo 2
          - Quantidade
          - Valor total
          - Custo médio


        Ativo 3
          - Quantidade
          - Valor total
          - Custo médio
        """,
        description="Agente que processa operações realizadas",
    )

    entrada_do_agente_de_processamento = types.Content(
        role="user",
        parts=[
            types.Part.from_text(text=f"Operações:{operacoes}"),
        ],
    )

    # Executa o agente
    dados_por_ativo = call_agent(agente_de_processamento_de_operacoes, entrada_do_agente_de_processamento)
    return dados_por_ativo

In [ ]:
######################################
# --- Agente 3: Busca dados complementares, como Tickers e CNPJ da empresa --- #
######################################
def agente_buscador_dados_complementares(ativos):
    buscador_dados_complementares = Agent(
        name="agente_buscador_dados_complementares",
        model=MODEL_ID_THINKING,
        instruction="""
            Você é especialista em negociar ativos na Bolsa de Valores, mas também é um especialista em declaração do IRPF,
            você trabalha em uma importante corretora e sabe muito de ativos negociados em bolsa.

            Você receberá uma lista de ativos, nesta lista tem alguns dados de cada ativo,  como uma descrição breve e quantidade, valor e preço médio.
            Busque dados complementares, como o ticker, o CNPJ da empresa e a Razão Social utilizando a tool (google_search) e complemente a lista de ativos.
            O ticker é uma informação muito importante, tente garantir que encontre o ticker para cada um dos ativos.

            Você tem grande conhecimento, se identificou que dois ativos são representados pelo mesmo Ticker, então são o mesmo ativo, então trate eles como o mesmo ativo
            e atualize seus dados de quantidade, valor total e preço médio.

            Também inclua em cada ativo a informação de (Sugestão de descrição do bem na declaração), onde irá apresentar a descrição do bem na ficha de bens e ativos, conforme as
            melhores práticas encontradas pela tool (google_search).

            Garanta que entre os dados de um ativo e outro, no resultado, exista pelo menos 2 linhas separando eles, garantindo que os dados não vão se misturar,
            para evitar confusão na leitura dos dados

            Utilize o formato abaixo na resposta:
            Ativo
              - Ticker
              - Razão Social
              - CNPJ
              - Descrição do bem na declaração
              - Sugestão de descrição do bem na declaração
              - Quantidade
              - Valor total
              - Custo médio



            """,
        description="Agente buscador de dados complementares",
        tools=[google_search]
    )

    entrada_do_agente_buscador_dados_complementares = types.Content(
        role="user",
        parts=[
            types.Part.from_text(text=f"Ativos:{ativos}"),
        ],
    )
    # Executa o agente
    rascunho = call_agent(buscador_dados_complementares, entrada_do_agente_buscador_dados_complementares)
    return rascunho

In [ ]:
print("🦁 Iniciando o cálculo de Preço Médio de seus ativos para facilitar sua declaração do IRPF 🦁")
print("-----------------------------------------------")
print("Selecione as notas de corretagem para a análise!")
print("-----------------------------------------------")

# --- Obtendo suas notas de corretagem ---
uploaded = files.upload()

for filename in uploaded:
    print(f'O arquivo "{filename}" com tamanho {len(uploaded[filename])} bytes será analisado.')

# --- Criando lista com nome dos arquivos ---
filenames = []
for filename in uploaded:
  filenames.append(filename)

# Inserir lógica do sistema de agentes ################################################
if isinstance(filenames, list) and len(filenames) > 0:
  print(f"Ok, vamos começar pelo mais chato 😩, vamos processar as notas de corretagem! Mas relaxa que quem vai trabalhar é o Gemini 😎 ")
  print("-----------------------------------------------")
  operacoes = agente_analisador_de_arquivos(uploaded)
  print("📈📉 Operações encontradas pelo agente analisador de arquivos de notas de corretagem! 📈📉")
  display(to_markdown(operacoes))
  print("-----------------------------------------------")

  print(f"Ok, agora que sabemos quais foram as operações, vamos processar os dados para calcular o preço médio de cada ativo! 💪 🤓 🧮 ➕➖✖️➗")
  print("-----------------------------------------------")
  ativos = agente_de_processamento_de_dados_de_operacoes(operacoes)
  print("💰 Agora já temos o preço médio de nossos ativos, quantidade e valor total, estamos quase lá! 💰")
  display(to_markdown(ativos))
  print("-----------------------------------------------")

  print(f"Agora vamos buscar alguns dados que ajudem na declaração de nossos Bens, como o ticker ( o \"código\" da ação ), CNPJ e outros dados.")
  print("Estamos na reta final!!! 🏁")
  print("-----------------------------------------------")
  ativos_com_dados_complementares = agente_buscador_dados_complementares(ativos)
  print("😊 Agora tenho os dados para fazer minha declaração de IRPF! 😊")
  display(to_markdown(ativos_com_dados_complementares))
  print("-----------------------------------------------")

  print("🧹 Agora vamos fazer uma limpeza e excluir os arquivos que foram enviados para o Colab, porque prezemos o ambiente limpo! 🗑️")
  for filename in filenames:
    command_rm_file = f"rm {filename}"
    os.system(command_rm_file)
    print(f"Arquivo {filename} excluído.")
else:
  print("Você esqueceu selecionar seus arquivos de notas de corregatem!")

